In [3]:
!pip install striprtf

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import json
import pandas as pd
import numpy as np
import sklearn

In [8]:
# Converting rtf file to text
from striprtf.striprtf import rtf_to_text

with open("C:/Users/Admin/Downloads/Dendrite/Screening Test - DS/algoparams_from_ui.json.rtf") as infile:
    content = infile.read()
    text = rtf_to_text(content)
print(type(text))

<class 'str'>


In [9]:
print(text)

{
    "session_name": "test",
    "session_description": "test",
    "design_state_data": {

      "session_info" : {
        "project_id": "1",
        "experiment_id": "kkkk-11",
        "dataset":"iris_modified.csv",
        "session_name": "test",
        "session_description": "test"
        },

      "target": {
        "prediction_type": "Regression",
        "target": "petal_width",
        "type":"regression",
        "partitioning": true
      },
      "train": {
        "policy": "Split the dataset",
        "time_variable": "sepal_length",
        "sampling_method": "No sampling(whole data)",
        "split": "Randomly",
        "k_fold": false,
        "train_ratio": 0,
        "random_seed": 0
      },
      "metrics": {
        "optomize_model_hyperparameters_for": "AUC",
        "optimize_threshold_for": "F1 Score",
        "compute_lift_at": 0,
        "cost_matrix_gain_for_true_prediction_true_result": 1,
        "cost_matrix_gain_for_true_prediction_false_result": 0,

In [10]:
# text is of string class we can parse it as json object

data = json.loads(text)
print(type(data))
print(data)

<class 'dict'>
{'session_name': 'test', 'session_description': 'test', 'design_state_data': {'session_info': {'project_id': '1', 'experiment_id': 'kkkk-11', 'dataset': 'iris_modified.csv', 'session_name': 'test', 'session_description': 'test'}, 'target': {'prediction_type': 'Regression', 'target': 'petal_width', 'type': 'regression', 'partitioning': True}, 'train': {'policy': 'Split the dataset', 'time_variable': 'sepal_length', 'sampling_method': 'No sampling(whole data)', 'split': 'Randomly', 'k_fold': False, 'train_ratio': 0, 'random_seed': 0}, 'metrics': {'optomize_model_hyperparameters_for': 'AUC', 'optimize_threshold_for': 'F1 Score', 'compute_lift_at': 0, 'cost_matrix_gain_for_true_prediction_true_result': 1, 'cost_matrix_gain_for_true_prediction_false_result': 0, 'cost_matrix_gain_for_false_prediction_true_result': 0, 'cost_matrix_gain_for_false_prediction_false_result': 0}, 'feature_handling': {'sepal_length': {'feature_name': 'sepal_length', 'is_selected': True, 'feature_vari

In [14]:
# Step1: get target variable
def get_target(json_data):

    # Step1:
    #  "target" variable comes under "design_state_data" section so let us first store target variable into one separate dictionary
    target = json_data['design_state_data']['target']
    prediction_type = target['prediction_type']

# Step2: Feature handling
def feature_handling(json_data, dataset):
    # Step2:
    feature_imputing = json_data['design_state_data']['feature_handling']
    for feature in feature_imputing:
        feature_property = feature_imputing[feature]
        feature_detail = feature_property['feature_details']
        if feature_detail.get('missing_values'):  #if the key named missing value is present then only we will impute the value to that feature column
            if feature_detail['impute_with'] == 'Average of values':
                col_mean = dataset[feature].mean() #mean of whole column
                dataset[feature].fillna(value=col_mean, inplace=True) #filling the null values with average value of the column
            elif feature_detail['impute_with'] == 'custom':
                dataset[feature].fillna(value=feature_detail['impute_value'], inplace=True) #filling the null value with custom value

# Step3: Feature reduction
# pass the dataset containing only training and target feature variable 
def feature_reduction(json_data, dataset):
    reduction_method_details = json_data['design_state_data']['feature_reduction']
    if reduction_method_details['feature_reduction_method'] == 'Tree-based':
        num_of_features = reduction_method_details['num_of_features_to_keep']
        n_trees = reduction_method_details['num_of_trees']
        tree_depth = reduction_method_details['depth_of_trees']

        X, y = sklearn.datasets.load_iris(return_X_y=True)
        tree_clf = sklearn.ensemble.RandomForestClassifier(n_estimators=n_trees, max_depth=tree_depth, max_features=num_of_features)
        tree_clf = tree_clf.fit(X, y)
        model = sklearn.feature_selection.SelectFromModel(tree_clf, prefit=True, max_features=num_of_features)
        X_new = model.transform(X)
        return X_new
    elif reduction_method_details['feature_reduction_method'] == 'Correlation with target':
        return dataset
    elif reduction_method_details['feature_reduction_method'] == 'PCA':
        return dataset
# Step 4 and 5
def get_parameters(json_data, dataset):
    X = dataset[['sepal_length','sepal_width','petal_length']]
    y = dataset['petal_width']
    parameters = json_data['design_state_data']['hyperparameters']
    random_state = parameters['random_state']
    parallelism = parameters['parallelism']
    n_folds = parameters['num_of_folds']

    models = json_data['design_state_data']['algorithms']
    fitted_model = []
    for m in models:
        if models[m]['is_selected']==True:
            estimator = f'sklearn.ensemble.{m}'
            model = sklearn.model_selection.GridSearchCV(estimator, scoring=None, n_jobs=parallelism, refit=True, cv=n_folds)
            model.fit(X,y)
            fitted_model.append(model)
    return fitted_model


In [11]:
iris_df = pd.read_csv("C:/Users/Admin/Downloads/Dendrite/Screening Test - DS/iris.csv")
iris_df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [12]:
iris_df['petal_width'].isnull().sum()

0